# Welcome
Solution Extraction is a process by which we take a Project Drawdown Solution, in the form of an Excel Workbook, and create a corresponding python solution that implements _most_ of the same functionality.  This notebook will guide you through that process.  See also `Extraction_Guide.md` for more explanation and notes.

The first step is _make a copy of this notebook_.  Give it a name that represents the model you will be working on.  That way it won't collide with other notebooks when you check in or merge fixes.

## Setup


In [3]:
from tools import solution_xls_extract as sxe
from tools import create_expected_zip as cez
from tools import expected_ghost
from tests import test_excel_integration as tei
from solution import factory
from pathlib import Path
import pandas as pd
import openpyxl
import importlib

In [4]:
# Identify where you will be storing your Excel file while you work on it, and what directory the final result will go into.

excelfile = Path("C:\\Users\\dkenny\\PycharmProjects\\solutions\\RecycledPlastics_RRS_Model_v1.1d.xlsm")
outdir = Path("C:\\Users\\dkenny\\PycharmProjects\\solutions\\solution\\recycledplastics")
outdir.mkdir(parents=True, exist_ok=True)

In [3]:
# If you make changes to the extraction code (or any other code), reload it
# NOTE: This kind of reloading DOES NOT work for solutions themselves, unfortunately.  If you re-generate or modify your solution,
# you have to restart the Jupyter kernel to get it to reload properly.

importlib.reload(sxe)

<module 'tools.solution_xls_extract' from 'C:\\Users\\dkenny\\PycharmProjects\\solutions\\tools\\solution_xls_extract.py'>

## Extract Code
Extraction is done by the `sxe.ouput_solution_python_file` function.  This function reads most of the data it needs to extract from the `ScenarioRecord` tab and additional data from the TAM, Adoption and other tabs, and writes them to a solution directory in the form of an `__init__.py` file and a bunch of csv and json files.  All of the solutions in `/solution` were produced this way.

In [4]:
# Expect to see some warnings from openpyxl; these can be ignored.  If there are other warnings, please note them, but they are not necessarily
# a problem.

sxe.output_solution_python_file(outputdir=outdir, xl_filename=str(excelfile))

C:\Users\dkenny\AppData\Local\Continuum\anaconda3\envs\pd-dev\lib\site-packages\openpyxl\reader\drawings.py:59: UserWarning: wmf image format is not supported so the image is being dropped
  warn(msg)
C:\Users\dkenny\AppData\Local\Continuum\anaconda3\envs\pd-dev\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
C:\Users\dkenny\AppData\Local\Continuum\anaconda3\envs\pd-dev\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\dkenny\AppData\Local\Continuum\anaconda3\envs\pd-dev\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [5]:
# %debug is your friend.  If the extraction fails with an exception, jump in and see if anything looks wrong

%debug

ERROR:root:No traceback has been produced, nothing to debug.


After you generate the `__init__.py` file, you'll need to modify two lines in it. First, on line 221, please change:
```
ht_ref_adoption_final = ref_tam_per_region.loc[2050] * (ht_ref_adoption_initial /
            ref_tam_per_region.loc[2014])
```
to
```
ht_ref_adoption_final = ref_tam_per_region.loc[2050] * (ht_ref_adoption_initial /
            ref_tam_per_region.loc[2018])
```
and then add `copy_pds_datapoint=False,` into the HelperTables constructor like so on line 241, changing this:
```
        self.ht = helpertables.HelperTables(ac=self.ac,
            ref_datapoints=ht_ref_datapoints, pds_datapoints=ht_pds_datapoints,
            pds_adoption_data_per_region=pds_adoption_data_per_region,
            ref_adoption_limits=ref_tam_per_region, pds_adoption_limits=pds_tam_per_region,
            use_first_pds_datapoint_main=False,
            adoption_base_year=2018,
            copy_pds_to_ref=False,
            pds_adoption_trend_per_region=pds_adoption_trend_per_region,
            pds_adoption_is_single_source=pds_adoption_is_single_source)
```
to:
```
        self.ht = helpertables.HelperTables(ac=self.ac,
            ref_datapoints=ht_ref_datapoints, pds_datapoints=ht_pds_datapoints,
            pds_adoption_data_per_region=pds_adoption_data_per_region,
            ref_adoption_limits=ref_tam_per_region, pds_adoption_limits=pds_tam_per_region,
            use_first_pds_datapoint_main=False,
            adoption_base_year=2018,
            copy_pds_to_ref=False,
            copy_pds_datapoint=False,
            pds_adoption_trend_per_region=pds_adoption_trend_per_region,
            pds_adoption_is_single_source=pds_adoption_is_single_source)
```

If you are working on one of the [Excel import issues](https://github.com/ProjectDrawdown/solutions/issues?q=is%3Aissue+is%3Aopen+label%3A%22Excel+Import%22), please add comments to it describing problems you run into.  And if you have found something that looks like a general problem, please [open a new issue](https://github.com/ProjectDrawdown/solutions/issues/new) for it on the github repo.

I can't overemphasize this: 
> **Finding, researching and reporting issues is hugely valuable for us, even if you don't fully solve them.**


## Load Code / Sniff Test

Once the code has been sucessfully extracted and placed into a directory in `solution/`, all the tools that work with solutions should become available.

In [4]:
# factory.one_solution_scenarios loads a single solution, by name.  It returns a constructor that can construct scenario objects
# for this solution, and a list of the scenario names.

(constructor,scenarios) = factory.one_solution_scenarios("recycledplastics")

In [5]:
# What scenarios did we get?

scenarios

['PDS1-25p2050-April2021', 'PDS2-57p2050-April2021', 'PDS3-92p2050-April2021']

In [6]:
# Let's build the 2nd one

myscenario = constructor(scenarios[1])

In [7]:
# %debug is your friend.

%debug

ERROR:root:No traceback has been produced, nothing to debug.


## Look at some results

TODO: it would be nice put some examples below, for example showing a little graph of something.

In [8]:
myscenario.c2.co2_mmt_reduced()

,World,OECD90,Eastern Europe,Asia (Sans Japan),Middle East and Africa,Latin America,China,India,EU,USA
Year,,,,,,,,,,
2014,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020,9.288238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021,11.855514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022,14.422791,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Create Test Results

**This step requires the Excel application, and thus only can be run on Windows or Mac.**

Create a clean temporary directory to generate the test set in.  Put (a copy of) your Excel spreadsheet in that directory.

Follow the instructions in `tools/CREATING_EXPECTED_ZIP.md` to create the CSV files in that directory.

In [4]:
# Run the VB macros first!

# Assemble the resulting csv files into the expected_zip file

csvdirectory = Path("C:\\Users\\dkenny\\PycharmProjects\\solutions\\tmp")
cez.create_expected_zip(csvdirectory)

In [13]:
# Move the resulting file where it belongs.

testdirectory = outdir / "testdata"
testdirectory.mkdir(exist_ok=True)

!cp $csvdirectory/expected.zip $testdirectory/expected.zip

The syntax of the command is incorrect.


## Create the Test

Now we are going to add your new solution to the testing infrastructure.  In an editor, open the file `tests/test_excel_integration.py`.
Scroll down to the bottom, copy one of the test functions there and modify it to fit your new model.  It will look something
like the following, where you subsitute `TESTNAME` with a unique name for your test, and `MODULE` with the name of your solution module (which is the same as the name of the directory it is in).

```
def test_<SOLUTIONNAME>_RRS():
    from solution import <MODULE>
    zipfilename = str(solutiondir.joinpath(
        '<MODULE>', 'testdata', 'expected.zip'))
    zip_f = zipfile.ZipFile(file=zipfilename)
    for scenario in <MODULE>.scenarios.keys():
        obj = <MODULE>.Scenario(scenario=scenario)
        verify = RRS_solution_verify_list(obj=obj, zip_f=zip_f)
        check_excel_against_object(
            obj=obj, zip_f=zip_f, scenario=scenario, verify=verify)
```

Be sure to note whether your model is an RSS model or a Land model and copy the right kind of test!

## Run the Test

The following shell command runs the test you just created (swapping in your solution name, of course)

In [5]:
# You could pytest from the shell, but it is a little more convenient to call the 
# test function directly:

tei.test_RecycledPlastics_RRS()

Checking PDS1-25p2050-April2021
TAM Data
Helper Tables
Emissions Factors
Unit Adoption Calculations
First Cost
Operating Cost
CO2 Calcs
CH4 Calcs
Checking PDS2-57p2050-April2021
TAM Data
Helper Tables
Emissions Factors
Unit Adoption Calculations
First Cost
Operating Cost
CO2 Calcs
CH4 Calcs
Checking PDS3-92p2050-April2021
TAM Data
Helper Tables
Emissions Factors
Unit Adoption Calculations
First Cost
Operating Cost
CO2 Calcs
CH4 Calcs


If the test throws an exception, you might be tempted to use %debug to look at it.  Unfortunately the main location where exceptions get thrown has already lost the context of the error.  Instead what you generally have to do is figure out where in the test suite the failure was and add a `breakpoint()` (also known as `import pdb; pdb.set_trace()` to old-schoolers) there, then run it again.   Hint: the error message will probably contain an Excel range, like B91:C137.  This is a good string to search for in `test_excel_integration.py`.

From there, you work your way back to the same questions we were working on above: is this a failure in extraction, model code, the excel workbook, or the test?  Rinse and repeat.

# Tips

## Don't forget to restart the Jupyter Notebook kernel if you have modified code

If you change code you need to either reload the library (the 3nd cell of this notebook) or restart the kernel.

## When comparing to Excel, make sure you've loaded the right Scenario

On the `ScenarioRecord` tab, cell `B9` shows the currently loaded scenario.  When a workbook is first opened, this is usally empty, meaning you don't know which
scenario was last loaded.  Select the scenario you are debugging against from the dropdown, and click on 'Load Scenario'.

## Beautifier for Excel Formulas

Are you looking at an excel formula with five nested `IF(...` expressions?  Try [https://www.excelformulabeautifier.com/](https://www.excelformulabeautifier.com/).  You're welcome.

# You Finished!
Did you get a clean test run?  Hurrah!  You've finished this import task.  Project Drawdown thanks you!